# GIAI ĐOẠN 1

# DỮ LIỆU NGÀNH HỌC

In [12]:
import requests
from bs4 import BeautifulSoup
import json
import os

# URL của trang danh mục ngành học
url = 'https://admission.tdtu.edu.vn/dai-hoc/nganh-hoc'

# Gửi yêu cầu GET đến trang web
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Danh sách lưu kết quả
result = []

# Tìm tất cả các nhóm ngành
groups = soup.find_all('div', class_='ts-ng-und')

for group in groups:
    # Lấy tên nhóm ngành
    group_name_tag = group.find(class_='ts-ng-und-td')
    if not group_name_tag:
        continue

    group_name = group_name_tag.get_text(strip=True)

    # Tìm tất cả các ngành trong nhóm
    majors = group.find_all('a')
    major_list = []
    for major in majors:
        major_name = major.get_text(strip=True)
        major_link = major['href']
        major_list.append({
            'name': major_name,
            'link': major_link
        })

    result.append({
        'group_name': group_name,
        'majors': major_list
    })

# Ghi ra file JSON
os.makedirs('phase1', exist_ok=True)
with open('phase1/tdtu_majors.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

print("✅ Đã xuất ra file tdtu_majors.json")

✅ Đã xuất ra file tdtu_majors.json


In [13]:
import os
import requests
from bs4 import BeautifulSoup
import json
import re

BASE_URL = 'https://admission.tdtu.edu.vn/dai-hoc/nganh-hoc'
OUTPUT_DIR = 'phase1'
DETAILS_DIR = 'phase1/details'

def get_text_safe(element):
    return element.get_text(strip=True) if element else ''

def crawl_major_detail(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    data = {}

    data['name'] = get_text_safe(soup.find('h2', class_='title'))

    description_block = soup.find('div', class_='block-nganh-1-content')
    data['description'] = '\n'.join(get_text_safe(p) for p in description_block.find_all('p')) if description_block else ''

    reasons = []
    for block in soup.select('div.gsc-column'):
        title = block.select_one('div.highlight_content > div.title')
        if title:
            reasons.append(get_text_safe(title))
    data['reasons'] = reasons

    data['programs'] = []

    # Tìm block chứa danh sách tab
    tabs_container = soup.select_one('div.gsc-tabs, div.nganh-link-admission')
    if tabs_container:
        tab_list = tabs_container.find('ul')
        if tab_list:
            for li in tab_list.find_all('li'):
                a = li.find('a')
                if a:
                    data['programs'].append({
                        'tab': get_text_safe(a),
                        'major_code': '',
                        'description': '',
                        'content': {}
                    })

    tab_contents = []
    if soup.select_one('div.gsc-tabs'):
        tab_contents = soup.select('div.gsc-tabs div.tab-content > .tab-pane')
    elif soup.select_one('div.nganh-link-admission'):
        # Tìm tất cả các <div> con trong .nganh-link-admission mà KHÔNG chứa <ul> (chỉ lấy content tab)
        nganh_container = soup.select_one('div.nganh-link-admission')
        tab_contents = [
            div for div in nganh_container.find_all('div', recursive=False)
            if not div.find('ul') and div.get('id')  # thường là #tieu-chuan, #tien-tien, ...
        ]

    for i, content in enumerate(tab_contents):
        if i >= len(data['programs']):
            continue
        program = data['programs'][i]
        tt_block = content.find('div', class_='tab-chuong-trinh-tt')
        if tt_block:
            strongs = [get_text_safe(s) for s in tt_block.find_all('strong') if get_text_safe(s)]
            full_text = ' '.join(strongs).strip()

            # Regex tìm tên ngành và mã ngành, ví dụ:
            # - Kỹ thuật phần mềm - Mã ngành: F7480103
            # - Luật (Chuyên ngành...) - Mã ngành: F 7380101
            match = re.search(r'^(.*?)\s*[-–—]?\s*Mã ngành[:：]?\s*([A-Z]{0,2}\s*\d{5,})', full_text, re.IGNORECASE)

            if match:
                program['name'] = match.group(1).strip()
                program['major_code'] = match.group(2).replace(" ", "").strip()  # Loại bỏ khoảng trắng
            else:
                program['name'] = full_text
                program['major_code'] = ''

        program['description'] = '\n'.join(get_text_safe(p) for p in content.find_all('p') if get_text_safe(p))
        ct_block = content.find('div', class_='tab-chuong-trinh')
        if ct_block:
            for section in ct_block.find_all('div', class_='tab-ct'):
                for flex in section.find_all('div', class_='tab-ct-flex'):
                    title = get_text_safe(flex.find('div', class_='tab-ct-flex-title'))
                    if title:
                        value = ' '.join(get_text_safe(p) for p in flex.find_all('p'))
                        program['content'][title] = value

    data['programs'] = [p for p in data['programs'] if p['tab'] and (p['description'] or p['content'])]
    data['images'] = [img['src'] for img in soup.select('div.gsc-image img') if img.get('src')]

    return data

# === MAIN ===
def main():
    os.makedirs(DETAILS_DIR, exist_ok=True)

    res = requests.get(BASE_URL)
    soup = BeautifulSoup(res.content, 'html.parser')

    result = []
    program_set = set()
    groups = soup.find_all('div', class_='ts-ng-und')

    for group in groups:
        group_name_tag = group.find(class_='ts-ng-und-td')
        if not group_name_tag:
            continue

        group_name = group_name_tag.get_text(strip=True)
        majors = group.find_all('a')
        major_list = []

        for major in majors:
            major_name = major.get_text(strip=True)
            major_link = major['href'].replace('../../../../dai-hoc/nganh-hoc', '').strip()
            full_url = BASE_URL + major_link

            info = crawl_major_detail(full_url)
            filename = major_link.strip('/').replace('/', '_') + '.json'
            detail_path = os.path.join('details', filename)
            full_detail_path = os.path.join(DETAILS_DIR, filename)

            with open(full_detail_path, 'w', encoding='utf-8') as f:
                json.dump(info, f, ensure_ascii=False, indent=2)

            for p in info.get('programs', []):
                if p.get('tab'):
                    program_set.add(p['tab'])

            major_list.append({
                'name': major_name,
                'link': major_link,
                'detail': detail_path
            })

        result.append({
            'group_name': group_name,
            'majors': major_list
        })

    with open(os.path.join(OUTPUT_DIR, 'tdtu_majors.json'), 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=2)

    with open(os.path.join(OUTPUT_DIR, 'programme_types.json'), 'w', encoding='utf-8') as f:
        json.dump(sorted(list(program_set)), f, ensure_ascii=False, indent=2)

    print("✅ Hoàn tất! Đã lưu vào thư mục 'output'")

if __name__ == '__main__':
    main()

✅ Hoàn tất! Đã lưu vào thư mục 'output'


In [14]:
import os
import json
import shutil

DETAILS_DIR = 'phase1/details'
OUTPUT_DIR = 'output'
TD_MAJORS_SRC = 'phase1/tdtu_majors.json'
PROG_TYPES_SRC = 'phase1/programme_types.json'
TD_MAJORS_DST = os.path.join(OUTPUT_DIR, 'tdtu_majors.json')
PROG_TYPES_DST = os.path.join(OUTPUT_DIR, 'programme_types.json')
MAJOR_DETAIL_OUT = os.path.join(OUTPUT_DIR, 'major-detail.jsonl')

def merge_major_details():
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    detail_files = [f for f in os.listdir(DETAILS_DIR) if f.endswith('.json')]
    detail_files.sort()  # Sắp xếp để dễ kiểm tra và nhất quán

    with open(MAJOR_DETAIL_OUT, 'w', encoding='utf-8') as out_f:
        for fname in detail_files:
            full_path = os.path.join(DETAILS_DIR, fname)
            with open(full_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                out_f.write(json.dumps(data, ensure_ascii=False) + '\n')
    print(f'Đã gom {len(detail_files)} file vào {MAJOR_DETAIL_OUT}')

def move_file(src, dst):
    if os.path.exists(src):
        shutil.move(src, dst)
        print(f'Đã chuyển {src} sang {dst}')
    else:
        print(f'Không tìm thấy file {src}')

def main():
    merge_major_details()
    move_file(TD_MAJORS_SRC, TD_MAJORS_DST)
    move_file(PROG_TYPES_SRC, PROG_TYPES_DST)

if __name__ == '__main__':
    main()

Đã gom 43 file vào output\major-detail.jsonl
Đã chuyển phase1/tdtu_majors.json sang output\tdtu_majors.json
Đã chuyển phase1/programme_types.json sang output\programme_types.json


# GIAI ĐOẠN 2

# PHƯƠNG THỨC TUYỂN SINH

In [15]:
import requests
from bs4 import BeautifulSoup
import os
import json

def extract_and_save(url_objs, json_dir="ptts", jsonl_dir="output"):
    os.makedirs(json_dir, exist_ok=True)
    os.makedirs(jsonl_dir, exist_ok=True)
    jsonl_path = os.path.join(jsonl_dir, "phuong-thuc.jsonl")
    with open(jsonl_path, "w", encoding="utf-8") as jsonl_file:
        for obj in url_objs:
            url = obj.get("url")
            name = obj.get("name", "output")
            try:
                print(f"Đang truy cập: {url}")
                resp = requests.get(url, timeout=20)
                resp.raise_for_status()
                soup = BeautifulSoup(resp.text, "html.parser")
                content_div = soup.find("div", id="page-main-content")
                if content_div:
                    # Thay từng thẻ <a> bằng (text)[url]
                    for a in content_div.find_all('a', href=True):
                        text = a.get_text(strip=True)
                        href = a['href']
                        replacement = f"({text})[{href}]"
                        a.replace_with(replacement)
                    # Lấy text thuần, loại bỏ HTML
                    text_content = content_div.get_text(separator="\n", strip=True)
                    # Tạo dict để lưu
                    data = {
                        "url": url,
                        "name": name,
                        "content": text_content
                    }
                    # Lưu file .json riêng trong ptts/
                    json_path = os.path.join(json_dir, f"{name}.json")
                    with open(json_path, "w", encoding="utf-8") as jf:
                        json.dump(data, jf, ensure_ascii=False, indent=2)
                    # Lưu vào .jsonl trong output/
                    jsonl_file.write(json.dumps(data, ensure_ascii=False) + "\n")
                    print(f"Đã lưu {json_path}")
                else:
                    print(f"Không tìm thấy div id='page-main-content' ở {url}")
            except Exception as e:
                print(f"Lỗi với {url}: {e}")

if __name__ == "__main__":
    urls = [
        {
            "url": "https://admission.tdtu.edu.vn/dai-hoc/tuyen-sinh/phuong-thuc-2025",
            "name": "phuong-thuc-2025"
        },
        {
            "url": "https://admission.tdtu.edu.vn/dai-hoc/tuyen-sinh/phuong-thuc-2024",
            "name": "phuong-thuc-2024"
        },
        {
            "url": "https://admission.tdtu.edu.vn/dai-hoc/tuyen-sinh/phuong-thuc-2023",
            "name": "phuong-thuc-2023"
        }
    ]
    extract_and_save(urls, json_dir="ptts", jsonl_dir="output")

Đang truy cập: https://admission.tdtu.edu.vn/dai-hoc/tuyen-sinh/phuong-thuc-2025
Đã lưu ptts\phuong-thuc-2025.json
Đang truy cập: https://admission.tdtu.edu.vn/dai-hoc/tuyen-sinh/phuong-thuc-2024
Đã lưu ptts\phuong-thuc-2024.json
Đang truy cập: https://admission.tdtu.edu.vn/dai-hoc/tuyen-sinh/phuong-thuc-2023
Đã lưu ptts\phuong-thuc-2023.json


# HỌC PHÍ HỌC BỔNG

In [16]:
import requests
from bs4 import BeautifulSoup
import os
import json

# Nội dung học bổng thủ công cho 2023
HOC_BONG_2023 = """
TRƯỜNG ĐẠI HỌC TÔN ĐỨC THẲNG
CHÍNH SÁCH HỌC BỔNG 2023
TDTU dành hơn 30 tỷ đồng cấp học bổng cho tân sinh viên khóa tuyền sinh 2023
100% học phí
năm học
2023 - 2024
Học bổng thủ khoa
Học bổng dành cho học sinh
các tỉnh hợp tác toàn diện
Học bồng dành cho
học sinh các trường ký kết
Học bổng chương trình
Đại học bằng tiếng Anh
Học bổng ngành thu hút
Học bổng ưu tiên xét tuyền
Học bổng chương trình
Liên kết đào tạo quốc tế
Học bổng
Phân hiệu Khánh Hòa
dành cho tân sinh viên có điểm xét tuyển cao nhất
dành cho học sinh thỏa điều kiện xét học bổng TDTU
của các tỉnh hợp tác toàn diện với Trường
(Bình Thuận, Gia Lai, Bình Định, Lâm Đồng, Quảng Ngãi)
dành cho học sinh các trường THPT có ký kết hợp tác
với TDTU thỏa điều kiện xét học bổng
100% tân sinh viện chương trình dự bị tiếng Anh,
chương trình đại học bằng tiếng Anh được cấp học bổng
theo điều kiện của TDTU
100% tân sinh viên các ngành thu hút
⚫ Công tác xã hội
Công nghệ Kỹ thuật môi trường
Khoa học môi trường
Bảo hộ lao động
Quy hoạch vùng và đô thị
Kỹ thuật xây dựng công trình giao thông
Quản lý thể dục thể thao - Chuyên ngành Golf
được cấp học bổng theo điều kiện của TDTU
100% tân sinh viên nhập học vào các ngành thu hút
theo phương thức ưu tiên xét tuyển của TDTU dành cho
các trường ký kết, có thư giới thiệu của Hiệu trưởng
trường THPT được xét cấp học bổng
100% tân sinh viên chương trình dự bị tiếng Anh,
chương trình Liên kết đào tạo quốc tế được xét cấp
học bồng theo điều kiện của TDTU
100% tân sinh viên học tại Phân hiệu Khánh Hòa
được xét cấp học bổng theo điều kiện của TDTU
Học bổng dành cho học sinh trường chuyên, trường trọng điểm
Học sinh giỏi 03 năm các trường THPT chuyên, trường trọng điểm thỏa điều kiện xét học bổng
Học bổng thế hệ đầu tiên học đại học
Tân sinh viên là thế hệ đầu tiên trong gia đình học đại học trúng tuyền vào các ngành thu hút được
xét cấp học bổng theo điều kiện của TDTU
Các loại học bổng khác
Học bổng cho anh/chị em ruột cùng học tại TDTU
Học bổng cho con, anh/chị/em ruột của cán bộ công đoàn
Học bổng cho sinh viên khuyết tật có hoàn cảnh khó khăn
Học bổng cho tân sinh viên có tiếng Anh đầu vào cao
và nhiều loại học bổng khác
"""

import requests
from bs4 import BeautifulSoup
import os
import json

def extract_tables_html(dd_or_div):
    # Trả về danh sách mã html của các bảng trong thẻ dd/div
    return [str(table) for table in dd_or_div.find_all("table")]

def extract_section_content_as_html(dd):
    html_parts = []
    for node in dd.children:
        if getattr(node, "name", None) == "table":
            html_parts.append(str(node))
        elif isinstance(node, str):
            t = node.strip()
            if t:
                html_parts.append(f"<p>{t}</p>")
        elif hasattr(node, "get_text") and node.name != "table":
            # Loại bỏ các bảng con để không lặp lại bảng
            clone = node.__copy__()
            for t in clone.find_all("table"):
                t.extract()
            t_html = str(clone)
            html_parts.append(t_html)
    return "\n".join(html_parts).strip()

def extract_hoc_phi(dl_tag):
    # Danh sách bảng ngoài dd (hiếm khi có)
    tables_outside_dd = [str(table)
                         for table in dl_tag.find_all("table")
                         if not table.find_parent("dd")]

    sections = []
    dts = dl_tag.find_all("dt")
    for dt in dts:
        section_title = dt.get_text(strip=True)
        dd = dt.find_next_sibling("dd")
        if not dd:
            continue
        section_html = extract_section_content_as_html(dd)
        sections.append({
            "title": section_title,
            "content": section_html
        })
    return sections, tables_outside_dd

def extract_hoc_bong(div):
    text = div.get_text(separator="\n", strip=True)
    tables_html = [str(table) for table in div.find_all("table")]
    # Ghép text với các bảng html
    content = text
    if tables_html:
        content += "\n" + "\n".join(tables_html)
    return content, tables_html

def extract_and_save(url_objs, output_path="output.jsonl"):
    with open(output_path, "w", encoding="utf-8") as out_f:
        for obj in url_objs:
            url = obj.get("url")
            name = obj.get("name", "output")
            try:
                print(f"Đang truy cập: {url}")
                resp = requests.get(url, timeout=20)
                resp.raise_for_status()
                soup = BeautifulSoup(resp.text, "html.parser")
                content_div = soup.find("div", id="page-main-content")
                if not content_div:
                    print(f"Không tìm thấy div id='page-main-content' ở {url}")
                    continue

                # Học phí
                hoc_phi_dl = content_div.find("dl")
                if hoc_phi_dl:
                    hoc_phi_sections, hoc_phi_tables = extract_hoc_phi(hoc_phi_dl)
                else:
                    hoc_phi_sections, hoc_phi_tables = [], []

                # Học bổng
                hoc_bong_div = content_div.find("div", class_="cshb")
                if hoc_bong_div:
                    hoc_bong_content, hoc_bong_tables = extract_hoc_bong(hoc_bong_div)
                else:
                    hoc_bong_content, hoc_bong_tables = "", []

                # Ngoại lệ cho 2023
                if "2023" in name and not hoc_bong_content.strip():
                    hoc_bong_content = HOC_BONG_2023.strip()
                    hoc_bong_tables = []

                json_obj = {
                    "url": url,
                    "name": name,
                    "hoc_phi": {
                        "sections": hoc_phi_sections,     # [{ "title":..., "content": <html string>}]
                        "tables": hoc_phi_tables          # [<table html>,...]
                    },
                    "hoc_bong": {
                        "content": hoc_bong_content,      # text + table html nối lại
                        "tables": hoc_bong_tables         # [<table html>,...]
                    }
                }
                out_f.write(json.dumps(json_obj, ensure_ascii=False) + "\n")
                print(f"Đã crawl và ghi vào {output_path}")

            except Exception as e:
                print(f"Lỗi với {url}: {e}")

if __name__ == "__main__":
    urls = [
        {
            "url": "https://admission.tdtu.edu.vn/hoc-tai-tdtu/hoc-phi-hoc-bong-2025",
            "name": "hoc-phi-hoc-bong-2025"
        },
        {
            "url": "https://admission.tdtu.edu.vn/hoc-tai-tdtu/hoc-phi-hoc-bong-2024",
            "name": "hoc-phi-hoc-bong-2024"
        },
        {
            "url": "https://admission.tdtu.edu.vn/hoc-tai-tdtu/hoc-phi-hoc-bong-2023",
            "name": "hoc-phi-hoc-bong-2023"
        }
    ]
    extract_and_save(urls, "output/hoc-phi-hoc-bong.jsonl")

Đang truy cập: https://admission.tdtu.edu.vn/hoc-tai-tdtu/hoc-phi-hoc-bong-2025
Đã crawl và ghi vào output/hoc-phi-hoc-bong.jsonl
Đang truy cập: https://admission.tdtu.edu.vn/hoc-tai-tdtu/hoc-phi-hoc-bong-2024
Đã crawl và ghi vào output/hoc-phi-hoc-bong.jsonl
Đang truy cập: https://admission.tdtu.edu.vn/hoc-tai-tdtu/hoc-phi-hoc-bong-2023
Đã crawl và ghi vào output/hoc-phi-hoc-bong.jsonl


# CHUẨN HÓA DATA

In [17]:
import json
import os
import re
import unicodedata
from typing import List, Dict, Any

def remove_accents(text: str) -> str:
    text = unicodedata.normalize('NFD', text)
    text = ''.join([c for c in text if unicodedata.category(c) != 'Mn'])
    return text

def slugify(text: str) -> str:
    text = remove_accents(text)
    text = text.strip().lower()
    text = re.sub(r"\W+", "_", text)
    text = re.sub(r"_+", "_", text)
    text = text.strip("_")
    return text.upper()

def load_jsonl(filename: str) -> List[Dict[str, Any]]:
    with open(filename, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f if line.strip()]

def load_json(filename: str) -> Any:
    with open(filename, 'r', encoding='utf-8') as f:
        return json.load(f)

def save_json(filename: str, data: Any) -> None:
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

def build_programme_title_to_id_map(programmes_json: str) -> Dict[str, str]:
    progs = load_json(programmes_json)
    mapping = {}
    for p in progs:
        if isinstance(p, dict):
            title = p['name'].split(":")[0].strip().lower()
            prog_id = p['id']
        else:
            title = p.split(":")[0].strip().lower()
            prog_id = slugify(title)
        mapping[title] = prog_id
    return mapping

def normalize_title(title: str) -> str:
    text = remove_accents(title.lower().strip())
    text = re.sub(r"\s+", " ", text)
    return text

def guess_programme_id(section_title: str, prog_map: Dict[str, str]) -> str:
    norm_title = normalize_title(section_title)
    for prog_title, prog_id in prog_map.items():
        if norm_title.startswith(remove_accents(prog_title)):
            return prog_id
    for prog_title, prog_id in prog_map.items():
        if remove_accents(prog_title) in norm_title:
            return prog_id
    return ""

def convert_tuitions_per_programme(
    hoc_phi_jsonl: str,
    programmes_json: str,
    years: List[int],
    out_path: str
) -> None:
    prog_map = build_programme_title_to_id_map(programmes_json)
    tuitions = []
    for doc in load_jsonl(hoc_phi_jsonl):
        year_match = re.search(r"(\d{4})", doc.get("name", ""))
        year = year_match.group(1) if year_match else ""
        hoc_phi = doc.get("hoc_phi", {})
        for section in hoc_phi.get("sections", []):
            prog_id = guess_programme_id(section.get("title", ""), prog_map)
            tuitions.append({
                "id": f"TUITION_{prog_id}_{year}" if prog_id else f"TUITION_UNKNOWN_{year}",
                "year_id": year,
                "programme_id": prog_id,
                "name": section.get("title", ""),
                "url": doc.get("url", ""),
                "content": section.get("content", "")  # Lấy mã html/text đã được crawl
            })
        # Nếu có bảng tổng quan ngoài dd, nối tất cả bảng html lại
        if hoc_phi.get("tables"):
            tuitions.append({
                "id": f"TUITION_OVERVIEW_{year}",
                "year_id": year,
                "programme_id": "",
                "name": f"Tổng quan học phí {year}",
                "url": doc.get("url", ""),
                "content": "\n".join(hoc_phi.get("tables", []))
            })
    save_json(out_path, tuitions)

def convert_scholarships(hoc_phi_jsonl: str, years: List[int], out_path: str) -> None:
    scholarships = []
    for doc in load_jsonl(hoc_phi_jsonl):
        year_match = re.search(r"(\d{4})", doc.get("name", ""))
        year = year_match.group(1) if year_match else ""
        hoc_bong = doc.get("hoc_bong", {})
        scholarships.append({
            "id": f"SCHOLARSHIPS_{year}",
            "year_id": year,
            "name": doc.get("name", "") + " - Học bổng",
            "url": doc.get("url", ""),
            "content": hoc_bong.get("content", "")  # Lấy text + bảng html đã được crawl
        })
    save_json(out_path, scholarships)

def convert_majors(major_detail_path: str, out_path: str) -> None:
    major_list = load_jsonl(major_detail_path)
    majors = [
        {
            "id": slugify(m["name"]),
            "name": m["name"],
            "description": m.get("description", "")
        }
        for m in major_list
    ]
    save_json(out_path, majors)

def convert_programmes(programme_types_path: str, out_path: str) -> None:
    progs = load_json(programme_types_path)
    result = [
        {
            "id": slugify(p.split(":")[0]),
            "name": p
        }
        for p in progs
    ]
    save_json(out_path, result)

def convert_years(years_list: List[int], out_path: str) -> None:
    years = [{"id": str(y), "name": str(y)} for y in years_list]
    save_json(out_path, years)

def convert_major_programmes_group_years(
    tdtu_majors_json: str,
    major_detail_jsonl: str,
    programme_types_json: str,
    years: List[int],
    out_major_path: str,
    out_major_programmes_path: str
) -> None:
    majors_data = load_json(tdtu_majors_json)
    major_detail = {slugify(m["name"]): m for m in load_jsonl(major_detail_jsonl)}
    progs = load_json(programme_types_json)
    prog_ids = {p: slugify(p.split(":")[0]) for p in progs}
    majors = []
    major_programmes = []

    for group in majors_data:
        for m in group.get("majors", []):
            major_id = slugify(m["name"])
            detail = major_detail.get(major_id, {})
            # ---- majors ----
            description = detail.get("description", "")
            reasons = detail.get("reasons", [])
            reasons_str = "\n".join(reasons) if reasons else ""
            images = detail.get("images", [])
            majors.append({
                "id": major_id,
                "name": m["name"],
                "description": description,
                "reasons": reasons_str,
                "images": json.dumps(images, ensure_ascii=False)
            })
            # ---- major_programmes ----
            # Mỗi object trong programs là một majorprogramme!
            programs = detail.get("programs", [])
            for prog in programs:
                tab = prog.get("tab", "")
                major_code = prog.get("major_code", "")
                programme_id = prog_ids.get(tab, "")
                mp_id = f"{major_id}_{programme_id}"
                node = {
                    "id": mp_id,
                    "major_id": major_id,
                    "programme_id": programme_id,
                    "tab": tab,
                    "major_code": major_code,
                    "description": prog.get("description", ""),
                    "name": prog.get("name", m["name"]),
                    "year_ids": [str(y) for y in years]
                }
                if "content" in prog and isinstance(prog["content"], dict):
                    for k, v in prog["content"].items():
                        node[k] = v
                major_programmes.append(node)

    save_json(out_major_path, majors)
    save_json(out_major_programmes_path, major_programmes)

def convert_documents_phuongthuc(phuong_thuc_jsonl: str, out_path: str) -> None:
    docs = []
    for doc in load_jsonl(phuong_thuc_jsonl):
        year_match = re.search(r"(\d{4})", doc.get("name", ""))
        year = year_match.group(1) if year_match else ""
        docs.append({
            "id": f"PHUONG_THUC_{year}",
            "year_id": year,
            "type": "phuong-thuc-tuyen-sinh",
            "name": doc.get("name", ""),
            "url": doc.get("url", ""),
            "html": doc.get("content", ""),
            "text": doc.get("content", "")
        })
    save_json(out_path, docs)

def main():
    data_dir = "./output"
    out_dir = "./out"
    os.makedirs(out_dir, exist_ok=True)

    convert_majors(
        os.path.join(data_dir, "major-detail.jsonl"),
        os.path.join(out_dir, "majors.json")
    )
    convert_programmes(
        os.path.join(data_dir, "programme_types.json"),
        os.path.join(out_dir, "programmes.json")
    )
    convert_years([2023, 2024, 2025], os.path.join(out_dir, "years.json"))

    convert_tuitions_per_programme(
        os.path.join(data_dir, "hoc-phi-hoc-bong.jsonl"),
        os.path.join(data_dir, "programme_types.json"),
        [2023, 2024, 2025],
        os.path.join(out_dir, "tuitions.json")
    )

    convert_scholarships(
        os.path.join(data_dir, "hoc-phi-hoc-bong.jsonl"),
        [2023, 2024, 2025],
        os.path.join(out_dir, "scholarships.json")
    )

    convert_documents_phuongthuc(
        os.path.join(data_dir, "phuong-thuc.jsonl"),
        os.path.join(out_dir, "documents.json")
    )

    convert_major_programmes_group_years(
        os.path.join(data_dir, "tdtu_majors.json"),
        os.path.join(data_dir, "major-detail.jsonl"),
        os.path.join(data_dir, "programme_types.json"),
        [2023, 2024, 2025],
        os.path.join(out_dir, "majors.json"),
        os.path.join(out_dir, "major_programmes.json")
    )

    print(f"All files converted to {out_dir}/")

if __name__ == "__main__":
    main()

All files converted to ./out/


# IMPORT

In [ ]:
import requests
import json
import os

API_BASE = "http://localhost:5000"
API_KEY = "TOKEN"

OUT_DIR = "./out"

def load_json(fname):
    with open(fname, encoding="utf-8") as f:
        return json.load(f)

def post_json(url, data):
    headers = {
        "Content-Type": "application/json"
    }
    headers["Authorization"] = f"Bearer {API_KEY}"
    resp = requests.post(url, headers=headers, json=data)
    try:
        resp.raise_for_status()
        print(f"POST {url} OK: {resp.status_code}")
        print(f"RES: {resp.json()}")
    except Exception as e:
        print(f"POST {url} FAILED: {e}")
        print(resp.text)

def import_majors_programmes_years():
    majors = load_json(os.path.join(OUT_DIR, "majors.json"))
    programmes = load_json(os.path.join(OUT_DIR, "programmes.json"))
    years = load_json(os.path.join(OUT_DIR, "years.json"))
    major_programmes = load_json(os.path.join(OUT_DIR, "major_programmes.json"))
    post_json(f"{API_BASE}/api/v2/import/majors-programmes-years", {
        "majors": majors,
        "programmes": programmes,
        "years": years,
        "major_programmes": major_programmes
    })

def import_tuitions():
    tuitions = load_json(os.path.join(OUT_DIR, "tuitions.json"))
    # Nếu cần truyền programmes, years, có thể load và gửi cùng data
    programmes = load_json(os.path.join(OUT_DIR, "programmes.json"))
    years = load_json(os.path.join(OUT_DIR, "years.json"))
    post_json(f"{API_BASE}/api/v2/import/tuitions", {
        "tuitions": tuitions,
        "programmes": programmes,
        "years": years
    })

def import_scholarships():
    scholarships = load_json(os.path.join(OUT_DIR, "scholarships.json"))
    years = load_json(os.path.join(OUT_DIR, "years.json"))
    post_json(f"{API_BASE}/api/v2/import/scholarships", {
        "scholarships": scholarships,
        "years": years
    })

def import_documents():
    documents = load_json(os.path.join(OUT_DIR, "documents.json"))
    years = load_json(os.path.join(OUT_DIR, "years.json"))
    post_json(f"{API_BASE}/api/v2/import/documents", {
        "documents": documents,
        "years": years
    })

def main():
    import_majors_programmes_years()
    import_tuitions()
    import_scholarships()
    import_documents()
    print("DONE!")

if __name__ == "__main__":
    main()